# QC-Py-08 - Multi-Asset Portfolio Strategies

> **Diversification, correlation et allocation tactique**
> Duree: 75 minutes | Niveau: Intermediaire-Avance | Python + QuantConnect

---

## Objectifs d'Apprentissage

A la fin de ce notebook, vous serez capable de :

1. Construire un **portfolio multi-classes d'actifs** (equities, bonds, commodities, crypto)
2. Analyser les **correlations** entre actifs et detecter les regimes de marche
3. Implementer plusieurs **strategies d'allocation** (equal weight, risk parity, tactical)
4. Comprendre les principes de **hedging** avec options et futures
5. Creer un **All-Weather Portfolio** inspire de Ray Dalio

## Prerequisites

- Notebooks QC-Py-01 a 07 completes
- Comprehension de base de la theorie de portefeuille
- Familiarite avec numpy et pandas

## Structure du Notebook

1. Introduction Multi-Asset (10 min)
2. Ajouter Multiple Asset Classes (10 min)
3. Calcul des Correlations (10 min)
4. Detection de Regime (10 min)
5. Equal Weight Portfolio (10 min)
6. Risk Parity (10 min)
7. Tactical Asset Allocation (10 min)
8. Hedging avec Options (10 min)
9. Hedging avec Futures (10 min)
10. All-Weather Portfolio (25 min)

---

## Partie 1 : Fondations Multi-Asset (20 min)

### 1.1 Pourquoi Diversifier ?

La diversification est souvent appelee "the only free lunch in finance" (Harry Markowitz). L'idee centrale est simple : en combinant des actifs qui ne bougent pas ensemble, on peut reduire le risque sans sacrifier le rendement.

#### Concepts Cles

| Concept | Definition | Importance |
|---------|-----------|------------|
| **Correlation** | Mesure de co-mouvement entre actifs (-1 a +1) | Faible correlation = meilleure diversification |
| **Risk-Adjusted Returns** | Rendement par unite de risque (ex: Sharpe Ratio) | Objectif : maximiser rendement/risque |
| **Modern Portfolio Theory** | Framework mathematique pour optimiser allocation | Frontiere efficiente, portfolio optimal |

#### Classes d'Actifs Principales

| Classe | Exemples QuantConnect | Caracteristiques |
|--------|----------------------|------------------|
| **Equities** | SPY, QQQ, IWM | Croissance, volatilite moyenne-haute |
| **Bonds** | TLT, IEF, BND | Revenus, faible volatilite, hedge contre recession |
| **Commodities** | GLD, USO, DBC | Couverture inflation, decorrelation |
| **Forex** | EURUSD, USDJPY | Liquidite, hedging devises |
| **Crypto** | BTCUSD, ETHUSD | Haute volatilite, potentiel croissance |

#### La Magie de la Diversification

Considerons deux actifs avec le meme rendement espere de 10% et volatilite de 20%.

| Correlation | Volatilite Portfolio 50/50 | Reduction Risque |
|------------|---------------------------|------------------|
| +1.0 | 20% | 0% |
| +0.5 | 17.3% | 13.5% |
| 0.0 | 14.1% | 29.5% |
| -0.5 | 10% | 50% |
| -1.0 | 0% | 100% |

> **Point cle** : Meme avec une correlation de 0, on reduit la volatilite de pres de 30% sans toucher au rendement espere !

### 1.2 Ajouter Multiple Asset Classes

Creons un portfolio diversifie avec QuantConnect. Nous utiliserons des ETFs comme proxies pour differentes classes d'actifs (accessibles aux investisseurs particuliers).

In [ ]:
# Imports QuantConnect
from AlgorithmImports import *

# Imports pour analyse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configuration matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Imports reussis")

In [ ]:
# Classe pour multi-asset portfolio
class MultiAssetPortfolioAlgorithm(QCAlgorithm):
    """
    Exemple de configuration multi-asset portfolio.
    Cette classe montre comment ajouter differentes classes d'actifs.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # === EQUITIES ===
        # Large-cap US (proxy S&P 500)
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        # Tech heavy (proxy Nasdaq)
        self.qqq = self.AddEquity("QQQ", Resolution.Daily).Symbol
        
        # === BONDS (via ETF) ===
        # Long-term Treasury (20+ years) - sensible aux taux
        self.tlt = self.AddEquity("TLT", Resolution.Daily).Symbol
        # Intermediate Treasury (7-10 years) - moins volatile
        self.ief = self.AddEquity("IEF", Resolution.Daily).Symbol
        
        # === COMMODITIES (via ETF) ===
        # Gold - valeur refuge, hedge inflation
        self.gld = self.AddEquity("GLD", Resolution.Daily).Symbol
        # Oil - cycle economique, energie
        self.uso = self.AddEquity("USO", Resolution.Daily).Symbol
        
        # === CRYPTO ===
        # Bitcoin - disponible via Coinbase/GDAX sur QuantConnect
        self.btc = self.AddCrypto("BTCUSD", Resolution.Daily).Symbol
        
        # Stocker tous les symboles pour faciliter les operations
        self.symbols = {
            'equities': [self.spy, self.qqq],
            'bonds': [self.tlt, self.ief],
            'commodities': [self.gld, self.uso],
            'crypto': [self.btc]
        }
        
        self.all_symbols = [self.spy, self.qqq, self.tlt, self.ief, 
                           self.gld, self.uso, self.btc]

# Afficher la structure
print("Structure Multi-Asset Portfolio:")
print("="*50)
print("\nEquities:")
print("  - SPY: S&P 500 ETF (large-cap US)")
print("  - QQQ: Nasdaq 100 ETF (tech-heavy)")
print("\nBonds:")
print("  - TLT: 20+ Year Treasury Bond ETF")
print("  - IEF: 7-10 Year Treasury Bond ETF")
print("\nCommodities:")
print("  - GLD: Gold ETF")
print("  - USO: United States Oil Fund")
print("\nCrypto:")
print("  - BTCUSD: Bitcoin (via Coinbase)")

> **Note technique** : 
> - Les ETFs (SPY, TLT, GLD, etc.) sont des "proxies" qui permettent aux investisseurs particuliers d'acceder a differentes classes d'actifs
> - La crypto est disponible via la bourse Coinbase/GDAX sur QuantConnect (tier gratuit)
> - En production, on pourrait aussi utiliser des futures pour commodities et bonds (plus efficient mais requiert plus de capital)

---

## Partie 2 : Correlations et Analyse (20 min)

### 2.1 Calcul des Correlations

La matrice de correlation nous permet de comprendre comment les actifs bougent les uns par rapport aux autres. C'est fondamental pour construire un portfolio bien diversifie.

In [ ]:
# Fonction pour calculer la matrice de correlation
def calculate_correlation_matrix(algorithm, symbols, lookback=60):
    """
    Calcule la matrice de correlation entre plusieurs actifs.
    
    Parameters:
    -----------
    algorithm : QCAlgorithm instance
    symbols : list of Symbol objects
    lookback : int, nombre de jours pour le calcul
    
    Returns:
    --------
    pd.DataFrame : matrice de correlation
    """
    # Recuperer l'historique
    history = algorithm.History(symbols, lookback, Resolution.Daily)
    
    # Extraire les prix de cloture
    closes = history['close'].unstack(level=0)
    
    # Calculer les returns journaliers
    returns = closes.pct_change().dropna()
    
    # Calculer la matrice de correlation
    correlation_matrix = returns.corr()
    
    return correlation_matrix, returns

print("Fonction calculate_correlation_matrix() definie")

In [ ]:
# Exemple d'utilisation dans un algorithme
class CorrelationAnalysisAlgorithm(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetCash(100000)
        
        # Ajouter actifs
        self.symbols = [
            self.AddEquity("SPY", Resolution.Daily).Symbol,
            self.AddEquity("TLT", Resolution.Daily).Symbol,
            self.AddEquity("GLD", Resolution.Daily).Symbol,
            self.AddEquity("QQQ", Resolution.Daily).Symbol,
        ]
        
        # Programmer calcul mensuel des correlations
        self.Schedule.On(
            self.DateRules.MonthStart("SPY"),
            self.TimeRules.AfterMarketOpen("SPY", 30),
            self.AnalyzeCorrelations
        )
    
    def AnalyzeCorrelations(self):
        """Analyse les correlations mensuellement."""
        history = self.History(self.symbols, 60, Resolution.Daily)
        
        if history.empty:
            return
        
        # Calculer returns
        closes = history['close'].unstack(level=0)
        returns = closes.pct_change().dropna()
        
        # Matrice de correlation
        corr_matrix = returns.corr()
        
        # Log les correlations importantes
        self.Debug(f"\n{self.Time.date()} - Correlation Matrix:")
        for i, sym1 in enumerate(self.symbols):
            for j, sym2 in enumerate(self.symbols):
                if i < j:  # Eviter doublons
                    corr = corr_matrix.iloc[i, j]
                    self.Debug(f"  {sym1.Value}/{sym2.Value}: {corr:.2f}")

print("CorrelationAnalysisAlgorithm defini")

### Visualisation de la Matrice de Correlation

En mode recherche (QuantBook), on peut visualiser la matrice de correlation avec une heatmap.

In [ ]:
# Exemple de visualisation (donnees simulees pour illustration)
# En production, utilisez qb.History() avec QuantBook

# Donnees de correlation typiques entre classes d'actifs
assets = ['SPY', 'QQQ', 'TLT', 'IEF', 'GLD', 'USO', 'BTC']
corr_data = np.array([
    [1.00, 0.92, -0.35, -0.20, 0.05, 0.40, 0.30],  # SPY
    [0.92, 1.00, -0.30, -0.15, 0.00, 0.35, 0.35],  # QQQ
    [-0.35, -0.30, 1.00, 0.85, 0.25, -0.10, -0.15], # TLT
    [-0.20, -0.15, 0.85, 1.00, 0.20, -0.05, -0.10], # IEF
    [0.05, 0.00, 0.25, 0.20, 1.00, 0.15, 0.20],    # GLD
    [0.40, 0.35, -0.10, -0.05, 0.15, 1.00, 0.25],  # USO
    [0.30, 0.35, -0.15, -0.10, 0.20, 0.25, 1.00],  # BTC
])

corr_df = pd.DataFrame(corr_data, index=assets, columns=assets)

# Visualisation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_df, annot=True, cmap='RdYlGn_r', center=0,
            vmin=-1, vmax=1, fmt='.2f', square=True,
            linewidths=0.5, cbar_kws={'label': 'Correlation'})
plt.title('Matrice de Correlation Multi-Asset\n(Donnees illustratives)', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nObservations cles:")
print("- SPY/QQQ: Tres correles (0.92) - meme classe d'actifs")
print("- SPY/TLT: Correlation negative (-0.35) - 'flight to quality'")
print("- GLD: Faible correlation avec tout - bon diversificateur")
print("- BTC: Correlation moderee - classe d'actifs emergente")

### 2.2 Rolling Correlations

Les correlations ne sont pas statiques. Elles evoluent dans le temps, notamment en periode de stress.

**Phenomenes observes** :
- En periode normale : correlations relativement stables
- En crise : correlations equities augmentent ("tout baisse ensemble")
- Flight to quality : equities down, bonds (TLT) up

In [ ]:
def calculate_rolling_correlation(returns_df, symbol1, symbol2, window=60):
    """
    Calcule la correlation glissante entre deux actifs.
    
    Parameters:
    -----------
    returns_df : pd.DataFrame avec returns de chaque actif
    symbol1, symbol2 : str, noms des colonnes
    window : int, fenetre de calcul en jours
    
    Returns:
    --------
    pd.Series : correlation glissante
    """
    return returns_df[symbol1].rolling(window).corr(returns_df[symbol2])

# Exemple avec donnees simulees
np.random.seed(42)
dates = pd.date_range('2020-01-01', '2023-12-31', freq='B')
n = len(dates)

# Simuler returns avec correlation variable
spy_returns = np.random.normal(0.0004, 0.012, n)
tlt_returns = -0.3 * spy_returns + np.random.normal(0.0001, 0.008, n)

# Ajouter periode de stress (correlation augmente en valeur absolue)
stress_start = int(n * 0.25)
stress_end = int(n * 0.30)
tlt_returns[stress_start:stress_end] = -0.7 * spy_returns[stress_start:stress_end] + np.random.normal(0.0001, 0.005, stress_end - stress_start)

returns_sim = pd.DataFrame({
    'SPY': spy_returns,
    'TLT': tlt_returns
}, index=dates)

# Calculer rolling correlation
rolling_corr = calculate_rolling_correlation(returns_sim, 'SPY', 'TLT', window=60)

# Visualisation
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Subplot 1: Returns cumules
cumulative = (1 + returns_sim).cumprod()
axes[0].plot(cumulative.index, cumulative['SPY'], label='SPY', color='navy', linewidth=1.5)
axes[0].plot(cumulative.index, cumulative['TLT'], label='TLT', color='darkorange', linewidth=1.5)
axes[0].axvspan(dates[stress_start], dates[stress_end], alpha=0.2, color='red', label='Periode stress')
axes[0].set_title('Returns Cumules SPY vs TLT', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Return Cumule')
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

# Subplot 2: Rolling correlation
axes[1].plot(rolling_corr.index, rolling_corr, color='purple', linewidth=1.5)
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1].axhline(y=rolling_corr.mean(), color='red', linestyle='--', 
                label=f'Moyenne: {rolling_corr.mean():.2f}')
axes[1].axvspan(dates[stress_start], dates[stress_end], alpha=0.2, color='red')
axes[1].set_title('Correlation Glissante 60 Jours (SPY-TLT)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Correlation')
axes[1].set_ylim(-1, 0.5)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nStatistiques Rolling Correlation SPY-TLT:")
print(f"  Moyenne: {rolling_corr.mean():.3f}")
print(f"  Min: {rolling_corr.min():.3f}")
print(f"  Max: {rolling_corr.max():.3f}")

### 2.3 Detection de Regime de Marche

On peut utiliser les correlations pour detecter le regime de marche et adapter l'allocation.

**Regimes typiques** :
- **Risk-On** : Equities montent, bonds stagnent/baissent, correlation SPY-TLT moderement negative
- **Risk-Off** : Equities baissent, bonds montent (flight to quality), correlation SPY-TLT fortement negative
- **Crise de liquidite** : Tout baisse (correlation positive transitoire)

In [ ]:
class RegimeDetectionAlgorithm(QCAlgorithm):
    """
    Algorithme qui detecte le regime de marche base sur les correlations
    et adapte l'allocation en consequence.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetCash(100000)
        
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.tlt = self.AddEquity("TLT", Resolution.Daily).Symbol
        self.gld = self.AddEquity("GLD", Resolution.Daily).Symbol
        
        self.symbols = [self.spy, self.tlt, self.gld]
        
        # Parametres detection regime
        self.correlation_window = 60
        self.stress_threshold = -0.5  # Correlation SPY-TLT < -0.5 = stress
        
        # Stocker l'historique des returns
        self.returns_history = {}
        self.current_regime = "normal"
        
        # Rebalancer mensuellement
        self.Schedule.On(
            self.DateRules.MonthStart("SPY"),
            self.TimeRules.AfterMarketOpen("SPY", 30),
            self.DetectRegimeAndRebalance
        )
    
    def DetectRegimeAndRebalance(self):
        """Detecte le regime et ajuste l'allocation."""
        
        # Recuperer historique
        history = self.History(self.symbols, self.correlation_window, Resolution.Daily)
        
        if history.empty or len(history) < self.correlation_window:
            return
        
        # Calculer returns
        closes = history['close'].unstack(level=0)
        returns = closes.pct_change().dropna()
        
        # Correlation SPY-TLT
        spy_returns = returns[self.spy]
        tlt_returns = returns[self.tlt]
        correlation = spy_returns.corr(tlt_returns)
        
        # Detecter regime
        if correlation < self.stress_threshold:
            regime = "stress"  # Flight to quality prononce
        elif correlation > 0.2:
            regime = "liquidity_crisis"  # Tout baisse ensemble (rare)
        else:
            regime = "normal"
        
        # Allocations selon regime
        allocations = {
            "normal": {self.spy: 0.60, self.tlt: 0.30, self.gld: 0.10},
            "stress": {self.spy: 0.30, self.tlt: 0.50, self.gld: 0.20},
            "liquidity_crisis": {self.spy: 0.20, self.tlt: 0.30, self.gld: 0.50}
        }
        
        # Appliquer allocation
        if regime != self.current_regime:
            self.Debug(f"{self.Time}: Regime change {self.current_regime} -> {regime} (corr: {correlation:.2f})")
            self.current_regime = regime
        
        for symbol, weight in allocations[regime].items():
            self.SetHoldings(symbol, weight)

print("RegimeDetectionAlgorithm defini")
print("\nRegimes et allocations:")
print("  Normal: SPY 60%, TLT 30%, GLD 10%")
print("  Stress: SPY 30%, TLT 50%, GLD 20%")
print("  Liquidity Crisis: SPY 20%, TLT 30%, GLD 50%")

> **Note pedagogique** : La detection de regime est un domaine actif de recherche. Cette implementation simple utilise uniquement la correlation SPY-TLT. Des approches plus sophistiquees utilisent des modeles Hidden Markov (HMM), des indicateurs de volatilite (VIX), ou du machine learning.

---

## Partie 3 : Strategies d'Allocation (30 min)

### 3.1 Equal Weight Portfolio

La strategie la plus simple : allouer le meme poids a chaque actif. Malgre sa simplicite, elle performe souvent bien grace a la diversification et au rebalancement implicite (vendre les gagnants, acheter les perdants).

In [ ]:
class EqualWeightPortfolioAlgorithm(QCAlgorithm):
    """
    Strategie Equal Weight avec rebalancement periodique.
    Allocation: 1/N pour chaque actif.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter actifs multi-classes
        tickers = ["SPY", "QQQ", "TLT", "IEF", "GLD", "USO"]
        self.symbols = [self.AddEquity(t, Resolution.Daily).Symbol for t in tickers]
        
        # Programmer rebalancement trimestriel
        self.Schedule.On(
            self.DateRules.MonthStart("SPY"),
            self.TimeRules.AfterMarketOpen("SPY", 30),
            self.Rebalance
        )
        
        self.rebalance_months = [1, 4, 7, 10]  # Trimestriel
    
    def Rebalance(self):
        """Rebalance vers equal weight."""
        
        # Verifier si c'est un mois de rebalancement
        if self.Time.month not in self.rebalance_months:
            return
        
        # Calculer poids egal
        weight = 1.0 / len(self.symbols)
        
        self.Debug(f"{self.Time.date()}: Rebalancing to equal weight ({weight:.2%} each)")
        
        # Appliquer allocation
        for symbol in self.symbols:
            self.SetHoldings(symbol, weight)

print("EqualWeightPortfolioAlgorithm defini")
print(f"\nAllocation: {100/6:.2f}% par actif")
print("Rebalancement: Trimestriel (Janvier, Avril, Juillet, Octobre)")

### 3.2 Risk Parity

L'idee du Risk Parity est d'allouer de sorte que chaque actif contribue egalement au risque total du portfolio. Cela signifie surponderer les actifs a faible volatilite (bonds) et sous-ponderer les actifs a haute volatilite (equities).

**Formule simplifiee** :
$$w_i = \frac{1/\sigma_i}{\sum_j 1/\sigma_j}$$

ou $\sigma_i$ est la volatilite de l'actif $i$.

In [ ]:
def risk_parity_weights(returns_df, annualize=True):
    """
    Calcule les poids Risk Parity bases sur la volatilite inverse.
    
    Parameters:
    -----------
    returns_df : pd.DataFrame avec returns de chaque actif en colonnes
    annualize : bool, si True annualise la volatilite (252 jours)
    
    Returns:
    --------
    pd.Series : poids normalises
    """
    # Calculer volatilite
    volatilities = returns_df.std()
    
    if annualize:
        volatilities = volatilities * np.sqrt(252)
    
    # Inverse volatilite
    inv_vol = 1 / volatilities
    
    # Normaliser pour que somme = 1
    weights = inv_vol / inv_vol.sum()
    
    return weights, volatilities

# Demonstration avec donnees simulees
np.random.seed(42)
n_days = 252

# Simuler returns avec volatilites differentes
returns_demo = pd.DataFrame({
    'SPY': np.random.normal(0.0004, 0.012, n_days),   # Vol ~19%
    'QQQ': np.random.normal(0.0005, 0.015, n_days),   # Vol ~24%
    'TLT': np.random.normal(0.0001, 0.008, n_days),   # Vol ~13%
    'IEF': np.random.normal(0.0001, 0.005, n_days),   # Vol ~8%
    'GLD': np.random.normal(0.0002, 0.010, n_days),   # Vol ~16%
})

# Calculer poids
weights, vols = risk_parity_weights(returns_demo)

# Affichage
print("Risk Parity Weights:")
print("="*50)
result_df = pd.DataFrame({
    'Volatilite Ann.': vols.apply(lambda x: f"{x:.1%}"),
    'Poids Equal Weight': [f"{1/5:.1%}"] * 5,
    'Poids Risk Parity': weights.apply(lambda x: f"{x:.1%}")
})
print(result_df.to_string())

# Visualisation comparative
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Equal Weight
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
axes[0].pie([0.2]*5, labels=weights.index, autopct='%1.0f%%', colors=colors)
axes[0].set_title('Equal Weight', fontsize=12, fontweight='bold')

# Risk Parity
axes[1].pie(weights, labels=weights.index, autopct='%1.0f%%', colors=colors)
axes[1].set_title('Risk Parity', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nObservation: Risk Parity surpondere les actifs a faible volatilite (IEF, TLT)")

In [ ]:
class RiskParityAlgorithm(QCAlgorithm):
    """
    Strategie Risk Parity: allouer en fonction de la volatilite inverse.
    Chaque actif contribue egalement au risque total.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter actifs
        tickers = ["SPY", "QQQ", "TLT", "IEF", "GLD"]
        self.symbols = [self.AddEquity(t, Resolution.Daily).Symbol for t in tickers]
        
        # Parametres
        self.lookback = 60  # Jours pour calcul volatilite
        
        # Rebalancement mensuel
        self.Schedule.On(
            self.DateRules.MonthStart("SPY"),
            self.TimeRules.AfterMarketOpen("SPY", 30),
            self.RebalanceRiskParity
        )
    
    def RiskParityWeights(self, symbols, lookback=60):
        """
        Calcule les poids Risk Parity bases sur la volatilite historique.
        """
        # Recuperer historique
        history = self.History(symbols, lookback, Resolution.Daily)
        
        if history.empty:
            # Fallback: equal weight
            return {s: 1.0/len(symbols) for s in symbols}
        
        # Extraire closes et calculer returns
        closes = history['close'].unstack(level=0)
        returns = closes.pct_change().dropna()
        
        # Volatilite annualisee
        volatilities = returns.std() * np.sqrt(252)
        
        # Inverse volatilite
        inv_vol = 1 / volatilities
        
        # Normaliser
        weights = inv_vol / inv_vol.sum()
        
        # Convertir en dict avec Symbol comme cle
        return {symbol: weights[symbol] for symbol in symbols if symbol in weights.index}
    
    def RebalanceRiskParity(self):
        """Rebalance selon les poids Risk Parity."""
        
        weights = self.RiskParityWeights(self.symbols, self.lookback)
        
        if not weights:
            return
        
        self.Debug(f"{self.Time.date()}: Risk Parity Rebalancing")
        for symbol, weight in weights.items():
            self.SetHoldings(symbol, weight)
            self.Debug(f"  {symbol.Value}: {weight:.2%}")

print("RiskParityAlgorithm defini")

### 3.3 Tactical Asset Allocation (TAA)

La Tactical Asset Allocation ajuste dynamiquement les poids en fonction de signaux de marche. L'approche la plus simple utilise le momentum (SMA 200).

**Regles** :
- Si prix > SMA 200 : actif en tendance haussiere -> allouer
- Si prix < SMA 200 : actif en tendance baissiere -> cash (ou autre actif refuge)

In [ ]:
class TacticalAssetAllocationAlgorithm(QCAlgorithm):
    """
    Tactical Asset Allocation avec signal momentum (SMA 200).
    Alloue uniquement aux actifs au-dessus de leur SMA 200.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter actifs
        tickers = ["SPY", "QQQ", "TLT", "GLD", "EFA"]  # EFA = MSCI EAFE
        self.symbols = []
        self.sma_indicators = {}
        
        for ticker in tickers:
            symbol = self.AddEquity(ticker, Resolution.Daily).Symbol
            self.symbols.append(symbol)
            
            # Creer indicateur SMA 200 pour chaque actif
            self.sma_indicators[symbol] = self.SMA(symbol, 200)
        
        # Warmup pour avoir les SMAs pretes
        self.SetWarmup(200)
        
        # Rebalancement mensuel
        self.Schedule.On(
            self.DateRules.MonthStart("SPY"),
            self.TimeRules.AfterMarketOpen("SPY", 30),
            self.TacticalRebalance
        )
    
    def TacticalRebalance(self):
        """Rebalance tactiquement selon le signal momentum."""
        
        if self.IsWarmingUp:
            return
        
        # Determiner quels actifs sont au-dessus de leur SMA
        active_symbols = []
        
        for symbol in self.symbols:
            sma = self.sma_indicators[symbol]
            
            if not sma.IsReady:
                continue
            
            current_price = self.Securities[symbol].Price
            
            if current_price > sma.Current.Value:
                active_symbols.append(symbol)
                self.Debug(f"  {symbol.Value}: ACTIVE (price {current_price:.2f} > SMA {sma.Current.Value:.2f})")
            else:
                self.Debug(f"  {symbol.Value}: INACTIVE (price {current_price:.2f} < SMA {sma.Current.Value:.2f})")
        
        # Calculer allocation
        if len(active_symbols) > 0:
            weight = 1.0 / len(active_symbols)
        else:
            weight = 0  # 100% cash si aucun actif actif
        
        self.Debug(f"{self.Time.date()}: TAA Rebalance - {len(active_symbols)} active assets")
        
        # Appliquer allocation
        for symbol in self.symbols:
            if symbol in active_symbols:
                self.SetHoldings(symbol, weight)
            else:
                self.Liquidate(symbol)  # Vendre si sous SMA

print("TacticalAssetAllocationAlgorithm defini")
print("\nLogique:")
print("  - Prix > SMA(200) -> Allouer (momentum positif)")
print("  - Prix < SMA(200) -> Cash (eviter tendance baissiere)")

> **Avantages TAA** :
> - Evite les grandes baisses (bear markets)
> - Suit les tendances de fond
>
> **Inconvenients** :
> - "Whipsaw" en marche lateraux (faux signaux)
> - Late entry/exit (le SMA est un indicateur retardataire)

---

## Partie 4 : Hedging (20 min)

### 4.1 Hedging avec Options

Les options permettent de proteger un portfolio contre les baisses tout en conservant le potentiel de hausse.

**Strategies principales** :
- **Protective Put** : Acheter un put pour limiter les pertes
- **Collar** : Protective put + covered call (finance le put avec le call)

In [ ]:
class ProtectivePutAlgorithm(QCAlgorithm):
    """
    Strategie Protective Put: long equity + long put pour protection.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter l'equity
        equity = self.AddEquity("SPY", Resolution.Minute)
        equity.SetDataNormalizationMode(DataNormalizationMode.Raw)
        self.spy = equity.Symbol
        
        # Ajouter options sur SPY
        option = self.AddOption("SPY", Resolution.Minute)
        self.option_symbol = option.Symbol
        
        # Filtre options: puts OTM, expiration 30-60 jours
        option.SetFilter(lambda universe: universe
            .IncludeWeeklys()
            .Strikes(-10, -2)   # OTM puts (strikes sous le prix actuel)
            .Expiration(30, 60))  # 30-60 jours d'expiration
        
        self.put_contract = None
    
    def OnData(self, data):
        # Verifier si on a des options disponibles
        if not self.Portfolio[self.spy].Invested:
            # Acheter SPY
            self.SetHoldings(self.spy, 0.90)  # 90% en SPY
        
        # Si pas de put actif, en acheter un
        if self.put_contract is None or not self.Portfolio[self.put_contract].Invested:
            self.BuyProtectivePut(data)
    
    def BuyProtectivePut(self, data):
        """Achete un put de protection."""
        
        # Recuperer la chaine d'options
        chain = data.OptionChains.get(self.option_symbol)
        
        if chain is None:
            return
        
        # Filtrer pour puts uniquement
        puts = [x for x in chain if x.Right == OptionRight.Put]
        
        if len(puts) == 0:
            return
        
        # Selectionner le put le plus proche de 5% OTM
        underlying_price = self.Securities[self.spy].Price
        target_strike = underlying_price * 0.95  # 5% OTM
        
        # Trouver le put le plus proche du target
        put = min(puts, key=lambda x: abs(x.Strike - target_strike))
        
        # Acheter le put
        qty = int(self.Portfolio[self.spy].Quantity / 100)  # 1 option = 100 shares
        if qty > 0:
            self.MarketOrder(put.Symbol, qty)
            self.put_contract = put.Symbol
            self.Debug(f"{self.Time}: Bought protective put - Strike: {put.Strike}, Expiry: {put.Expiry}")

print("ProtectivePutAlgorithm defini")
print("\nStrategic Protective Put:")
print("  - Long 100 shares SPY")
print("  - Long 1 put SPY (5% OTM, 30-60 DTE)")
print("  - Protection: pertes limitees a ~5% + prime du put")

In [ ]:
class CollarStrategyAlgorithm(QCAlgorithm):
    """
    Strategie Collar: Protective Put + Covered Call.
    La prime du call finance (partiellement) le put.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Setup equity et options
        equity = self.AddEquity("SPY", Resolution.Minute)
        equity.SetDataNormalizationMode(DataNormalizationMode.Raw)
        self.spy = equity.Symbol
        
        option = self.AddOption("SPY", Resolution.Minute)
        self.option_symbol = option.Symbol
        
        # Filtre: strikes autour du prix actuel
        option.SetFilter(lambda universe: universe
            .IncludeWeeklys()
            .Strikes(-5, 5)
            .Expiration(30, 60))
        
        self.collar_active = False
    
    def OnData(self, data):
        # Etablir position equity si necessaire
        if not self.Portfolio[self.spy].Invested:
            shares = int(self.Portfolio.Cash * 0.9 / self.Securities[self.spy].Price)
            shares = (shares // 100) * 100  # Arrondir a 100
            if shares >= 100:
                self.MarketOrder(self.spy, shares)
        
        # Mettre en place le collar si pas encore actif
        if not self.collar_active and self.Portfolio[self.spy].Quantity >= 100:
            self.SetupCollar(data)
    
    def SetupCollar(self, data):
        """Met en place le collar: long put + short call."""
        
        chain = data.OptionChains.get(self.option_symbol)
        if chain is None:
            return
        
        underlying_price = self.Securities[self.spy].Price
        
        # Separer puts et calls
        puts = [x for x in chain if x.Right == OptionRight.Put]
        calls = [x for x in chain if x.Right == OptionRight.Call]
        
        if len(puts) == 0 or len(calls) == 0:
            return
        
        # Put: 5% OTM (protection)
        put_target = underlying_price * 0.95
        selected_put = min(puts, key=lambda x: abs(x.Strike - put_target))
        
        # Call: 5% OTM (cap le gain)
        call_target = underlying_price * 1.05
        selected_call = min(calls, key=lambda x: abs(x.Strike - call_target))
        
        # Nombre de contrats
        qty = int(self.Portfolio[self.spy].Quantity / 100)
        
        if qty > 0:
            # Acheter put (protection)
            self.MarketOrder(selected_put.Symbol, qty)
            # Vendre call (finance le put, mais cap le gain)
            self.MarketOrder(selected_call.Symbol, -qty)
            
            self.collar_active = True
            self.Debug(f"{self.Time}: Collar established")
            self.Debug(f"  Long Put: Strike {selected_put.Strike}, Premium {selected_put.LastPrice:.2f}")
            self.Debug(f"  Short Call: Strike {selected_call.Strike}, Premium {selected_call.LastPrice:.2f}")

print("CollarStrategyAlgorithm defini")
print("\nStrategie Collar:")
print("  - Long SPY")
print("  - Long Put 5% OTM (protection downside)")
print("  - Short Call 5% OTM (finance le put, cap le gain)")
print("\nResultat: Pertes limitees a ~5%, Gains limites a ~5%")

### 4.2 Hedging avec Futures

Les futures permettent de hedger un portfolio equity en vendant des contrats futures sur indice (ex: ES pour S&P 500).

**Calcul du Hedge Ratio** :
$$\text{Contracts} = \frac{\text{Portfolio Value} \times \beta}{\text{Futures Value}}$$

ou :
- $\beta$ = beta du portfolio vs l'indice (mesure la sensibilite)
- Futures Value = Prix du future x multiplicateur (50 pour ES)

In [ ]:
class FuturesHedgingAlgorithm(QCAlgorithm):
    """
    Hedging d'un portfolio equity avec futures ES (S&P 500 E-mini).
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(500000)  # Plus de capital pour marges futures
        
        # Portfolio equity
        self.equities = [
            self.AddEquity("AAPL", Resolution.Daily).Symbol,
            self.AddEquity("MSFT", Resolution.Daily).Symbol,
            self.AddEquity("GOOGL", Resolution.Daily).Symbol,
            self.AddEquity("AMZN", Resolution.Daily).Symbol,
        ]
        
        # Futures ES (S&P 500 E-mini)
        self.es = self.AddFuture(Futures.Indices.SP500EMini, Resolution.Daily)
        self.es.SetFilter(0, 90)  # Contrats expirant dans 0-90 jours
        
        # Parametres hedging
        self.portfolio_beta = 1.2  # Beta estime du portfolio (tech-heavy = beta > 1)
        self.es_multiplier = 50    # 1 point ES = $50
        self.hedge_ratio = 0.5     # Hedger 50% du portfolio
        
        self.hedge_active = False
        self.hedge_contract = None
        
        # Programmer rebalancement
        self.Schedule.On(
            self.DateRules.MonthStart("AAPL"),
            self.TimeRules.AfterMarketOpen("AAPL", 30),
            self.RebalanceHedge
        )
    
    def OnData(self, data):
        # Etablir position equity si necessaire
        if not any(self.Portfolio[s].Invested for s in self.equities):
            weight = 0.8 / len(self.equities)  # 80% en equities, 20% reserve pour marges
            for symbol in self.equities:
                self.SetHoldings(symbol, weight)
    
    def CalculateHedgeContracts(self):
        """
        Calcule le nombre de contrats ES a shorter pour hedger le portfolio.
        """
        # Valeur du portfolio equity
        equity_value = sum(self.Portfolio[s].HoldingsValue for s in self.equities)
        
        if equity_value <= 0:
            return 0
        
        # Valeur a hedger
        value_to_hedge = equity_value * self.hedge_ratio
        
        # Ajuster par le beta
        beta_adjusted_value = value_to_hedge * self.portfolio_beta
        
        # Trouver le contrat ES actif
        if self.hedge_contract is None:
            return 0
        
        # Valeur d'un contrat ES
        es_price = self.Securities[self.hedge_contract].Price
        es_contract_value = es_price * self.es_multiplier
        
        # Nombre de contrats (negatif car on short)
        contracts = -int(beta_adjusted_value / es_contract_value)
        
        return contracts
    
    def RebalanceHedge(self):
        """Rebalance la position de hedge."""
        
        # Trouver le contrat ES front-month
        for chain in self.CurrentSlice.FutureChains:
            contracts = list(chain.Value)
            if len(contracts) > 0:
                # Prendre le contrat le plus proche de l'expiration
                front_month = sorted(contracts, key=lambda x: x.Expiry)[0]
                self.hedge_contract = front_month.Symbol
                break
        
        if self.hedge_contract is None:
            self.Debug("No ES contract found for hedging")
            return
        
        # Calculer contracts necessaires
        target_contracts = self.CalculateHedgeContracts()
        current_contracts = int(self.Portfolio[self.hedge_contract].Quantity)
        
        # Ajuster position
        delta = target_contracts - current_contracts
        
        if delta != 0:
            self.MarketOrder(self.hedge_contract, delta)
            self.Debug(f"{self.Time.date()}: Hedge adjusted by {delta} contracts")
            self.Debug(f"  Portfolio Value: ${sum(self.Portfolio[s].HoldingsValue for s in self.equities):,.0f}")
            self.Debug(f"  ES Contracts: {target_contracts}")

print("FuturesHedgingAlgorithm defini")
print("\nParametres de hedge:")
print(f"  Portfolio Beta: 1.2 (tech-heavy)")
print(f"  ES Multiplier: $50 par point")
print(f"  Hedge Ratio: 50% du portfolio")
print("\nFormule: Contracts = (Portfolio Value * Beta * Hedge Ratio) / (ES Price * 50)")

> **Note importante sur les Futures** :
> - Les futures requierent des marges (capital immobilise)
> - Le multiplicateur ES est $50 (1 contrat ES = $50 x prix SP500)
> - Un contrat ES mini (/ES) represente ~$200k de valeur notionnelle
> - Pour les petits portfolios, considerer les micro futures (/MES, multiplicateur $5)

---

## Partie 5 : All-Weather Portfolio (25 min)

### 5.1 Concept du All-Weather Portfolio

Le All-Weather Portfolio est une strategie developpee par Ray Dalio (Bridgewater Associates). L'idee est de creer un portfolio qui performe dans tous les environnements economiques.

**Les 4 Scenarios Economiques** :

| Environnement | Croissance | Inflation | Actifs Favorables |
|--------------|------------|-----------|-------------------|
| **Rising Growth** | Hausse | - | Equities, Corporate Bonds, Commodities |
| **Falling Growth** | Baisse | - | Treasury Bonds, TIPS |
| **Rising Inflation** | - | Hausse | Commodities, TIPS, Gold |
| **Falling Inflation** | - | Baisse | Equities, Treasury Bonds |

**Allocation Originale Dalio** :
- 30% Actions (SPY)
- 40% Obligations Long Terme (TLT)
- 15% Obligations Moyen Terme (IEF)
- 7.5% Or (GLD)
- 7.5% Commodities (DBC ou GSG)

In [ ]:
# Visualisation de l'allocation All-Weather
allocations = {
    'Equities (SPY)': 30,
    'Long-Term Bonds (TLT)': 40,
    'Intermediate Bonds (IEF)': 15,
    'Gold (GLD)': 7.5,
    'Commodities (DBC)': 7.5
}

# Couleurs par classe d'actifs
colors = ['#1f77b4', '#ff7f0e', '#ffbb78', '#ffd700', '#8c564b']

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
axes[0].pie(allocations.values(), labels=allocations.keys(), autopct='%1.1f%%',
            colors=colors, explode=[0.02]*5, shadow=True)
axes[0].set_title('All-Weather Portfolio\n(Ray Dalio)', fontsize=14, fontweight='bold')

# Bar chart par classe
classes = ['Equities', 'Bonds', 'Commodities']
class_weights = [30, 55, 15]  # 30% eq, 55% bonds (40+15), 15% commodities (7.5+7.5)
class_colors = ['#1f77b4', '#ff7f0e', '#8c564b']

bars = axes[1].bar(classes, class_weights, color=class_colors, edgecolor='black')
axes[1].set_ylabel('Allocation (%)', fontsize=12)
axes[1].set_title('Allocation par Classe d\'Actifs', fontsize=14, fontweight='bold')
axes[1].set_ylim(0, 60)

# Ajouter valeurs sur les barres
for bar, val in zip(bars, class_weights):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{val}%', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nPhilosophie All-Weather:")
print("  - Surponderation bonds (55%) : stabilite, revenus reguliers")
print("  - Equities modere (30%) : croissance long terme")
print("  - Commodities (15%) : protection inflation, diversification")

### 5.2 Implementation Complete

In [ ]:
class AllWeatherPortfolioAlgorithm(QCAlgorithm):
    """
    All-Weather Portfolio inspire de Ray Dalio.
    
    Allocation:
    - 30% Equities (SPY)
    - 40% Long-Term Bonds (TLT)
    - 15% Intermediate Bonds (IEF)
    - 7.5% Gold (GLD)
    - 7.5% Commodities (DBC)
    
    Features:
    - Rebalancement trimestriel
    - Risk monitoring (volatilite, correlations)
    - Alertes si correlations changent significativement
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # === CONFIGURATION PORTFOLIO ===
        self.target_allocations = {}
        
        # Equities
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.target_allocations[self.spy] = 0.30
        
        # Long-Term Bonds
        self.tlt = self.AddEquity("TLT", Resolution.Daily).Symbol
        self.target_allocations[self.tlt] = 0.40
        
        # Intermediate Bonds
        self.ief = self.AddEquity("IEF", Resolution.Daily).Symbol
        self.target_allocations[self.ief] = 0.15
        
        # Gold
        self.gld = self.AddEquity("GLD", Resolution.Daily).Symbol
        self.target_allocations[self.gld] = 0.075
        
        # Commodities (DBC = Invesco DB Commodity Index)
        self.dbc = self.AddEquity("DBC", Resolution.Daily).Symbol
        self.target_allocations[self.dbc] = 0.075
        
        self.symbols = list(self.target_allocations.keys())
        
        # === PARAMETRES RISK MONITORING ===
        self.correlation_lookback = 60  # Jours pour calcul correlation
        self.correlation_threshold = 0.3  # Seuil alerte changement correlation
        self.baseline_correlations = {}  # Correlations de reference
        
        # === REBALANCEMENT TRIMESTRIEL ===
        self.rebalance_months = [1, 4, 7, 10]  # Janvier, Avril, Juillet, Octobre
        
        self.Schedule.On(
            self.DateRules.MonthStart("SPY"),
            self.TimeRules.AfterMarketOpen("SPY", 30),
            self.MonthlyCheck
        )
        
        # === WARMUP ===
        self.SetWarmup(self.correlation_lookback)
    
    def MonthlyCheck(self):
        """Verification mensuelle: rebalancement et monitoring."""
        
        if self.IsWarmingUp:
            return
        
        # Risk monitoring (tous les mois)
        self.MonitorRisk()
        
        # Rebalancement (trimestriel)
        if self.Time.month in self.rebalance_months:
            self.Rebalance()
    
    def Rebalance(self):
        """Rebalance vers les allocations cibles."""
        
        self.Debug(f"\n{'='*50}")
        self.Debug(f"{self.Time.date()}: QUARTERLY REBALANCE")
        self.Debug(f"{'='*50}")
        
        total_value = self.Portfolio.TotalPortfolioValue
        self.Debug(f"Portfolio Value: ${total_value:,.2f}")
        
        # Calculer drift (ecart par rapport a la cible)
        self.Debug(f"\nAllocations (Current -> Target):")
        
        for symbol in self.symbols:
            current_value = self.Portfolio[symbol].HoldingsValue
            current_weight = current_value / total_value if total_value > 0 else 0
            target_weight = self.target_allocations[symbol]
            drift = current_weight - target_weight
            
            self.Debug(f"  {symbol.Value}: {current_weight:.1%} -> {target_weight:.1%} (drift: {drift:+.1%})")
            
            # Appliquer allocation cible
            self.SetHoldings(symbol, target_weight)
        
        self.Debug(f"\nRebalance complete.")
    
    def MonitorRisk(self):
        """Surveille les metriques de risque et alerte si changement significatif."""
        
        # Recuperer historique
        history = self.History(self.symbols, self.correlation_lookback, Resolution.Daily)
        
        if history.empty or len(history) < self.correlation_lookback:
            return
        
        # Calculer returns et correlations
        closes = history['close'].unstack(level=0)
        returns = closes.pct_change().dropna()
        
        # Volatilite portfolio
        weights = np.array([self.target_allocations[s] for s in self.symbols])
        portfolio_returns = returns.dot(weights)
        portfolio_vol = portfolio_returns.std() * np.sqrt(252)
        
        # Correlation SPY-TLT (indicateur cle)
        spy_tlt_corr = returns[self.spy].corr(returns[self.tlt])
        
        # Matrice de correlation complete
        corr_matrix = returns.corr()
        
        # Stocker comme baseline si premier calcul
        key = "spy_tlt"
        if key not in self.baseline_correlations:
            self.baseline_correlations[key] = spy_tlt_corr
            self.Debug(f"{self.Time.date()}: Baseline SPY-TLT correlation set to {spy_tlt_corr:.2f}")
        
        # Verifier changement significatif
        baseline = self.baseline_correlations[key]
        change = abs(spy_tlt_corr - baseline)
        
        if change > self.correlation_threshold:
            self.Debug(f"\n*** ALERT: SPY-TLT correlation changed significantly ***")
            self.Debug(f"  Baseline: {baseline:.2f}")
            self.Debug(f"  Current: {spy_tlt_corr:.2f}")
            self.Debug(f"  Change: {change:.2f}")
            
            # Envoyer notification (en production, email/SMS/etc.)
            self.Notify.Email("trader@example.com", 
                             "All-Weather Alert: Correlation Shift",
                             f"SPY-TLT correlation changed from {baseline:.2f} to {spy_tlt_corr:.2f}")
            
            # Mettre a jour baseline
            self.baseline_correlations[key] = spy_tlt_corr
        
        # Log mensuel
        self.Debug(f"\n{self.Time.date()}: RISK MONITOR")
        self.Debug(f"  Portfolio Vol (ann.): {portfolio_vol:.1%}")
        self.Debug(f"  SPY-TLT Correlation: {spy_tlt_corr:.2f}")
    
    def OnEndOfAlgorithm(self):
        """Resume final."""
        self.Debug(f"\n{'='*50}")
        self.Debug("ALL-WEATHER PORTFOLIO - FINAL SUMMARY")
        self.Debug(f"{'='*50}")
        self.Debug(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:,.2f}")
        self.Debug(f"Total Return: {(self.Portfolio.TotalPortfolioValue / 100000 - 1) * 100:.2f}%")

print("AllWeatherPortfolioAlgorithm defini")

In [ ]:
# Resume de la strategie All-Weather
print("="*60)
print("ALL-WEATHER PORTFOLIO - RESUME")
print("="*60)

print("\n1. ALLOCATION CIBLE:")
print("   - SPY (Equities):      30%")
print("   - TLT (Long Bonds):    40%")
print("   - IEF (Int. Bonds):    15%")
print("   - GLD (Gold):          7.5%")
print("   - DBC (Commodities):   7.5%")

print("\n2. REBALANCEMENT:")
print("   - Frequence: Trimestriel (Jan, Avr, Jul, Oct)")
print("   - Methode: Reequilibrer vers allocations cibles")

print("\n3. RISK MONITORING:")
print("   - Volatilite portfolio (annualisee)")
print("   - Correlation SPY-TLT (indicateur cle)")
print("   - Alerte si changement > 0.3 en correlation")

print("\n4. AVANTAGES:")
print("   - Robuste dans differents environnements economiques")
print("   - Faible volatilite (surponderation bonds)")
print("   - Diversification reelle (actifs decorrelated)")

print("\n5. INCONVENIENTS:")
print("   - Performance moderate en bull markets")
print("   - Sensible aux taux d'interet (40% TLT)")
print("   - Surponderation historiquement en bonds")

---

## Conclusion et Prochaines Etapes

### Recapitulatif

Dans ce notebook, nous avons couvert :

1. **Fondations Multi-Asset** :
   - Pourquoi diversifier (correlation, risk-adjusted returns)
   - Classes d'actifs (equities, bonds, commodities, crypto)
   - Comment ajouter differentes classes dans QuantConnect

2. **Analyse des Correlations** :
   - Matrice de correlation statique
   - Rolling correlations pour detecter les changements
   - Detection de regime (normal, stress, liquidity crisis)

3. **Strategies d'Allocation** :
   - Equal Weight (simplicite, diversification naturelle)
   - Risk Parity (contribution egale au risque)
   - Tactical Asset Allocation (momentum SMA 200)

4. **Hedging** :
   - Protective puts et collars (options)
   - Hedge avec futures ES (beta-adjusted)

5. **All-Weather Portfolio** :
   - Philosophie Ray Dalio
   - Implementation complete avec risk monitoring

### Points Cles a Retenir

| Concept | Point Cle |
|---------|----------|
| **Diversification** | Correlation < 1 = reduction risque sans perte de rendement |
| **Risk Parity** | Allouer par contribution au risque, pas par capital |
| **TAA** | Momentum simple (SMA 200) peut eviter les bear markets |
| **Hedging** | Options = protection avec upside, Futures = hedge lineaire |
| **All-Weather** | Surponderer bonds pour stabilite dans tous regimes |

### Limitations

- **ETFs comme proxies** : Les ETFs ont des frais et tracking error vs actifs sous-jacents
- **Correlations non-stationnaires** : Les correlations changent, surtout en crise
- **Covariance estimation** : Difficile avec peu de donnees (regime shifts)
- **Leverage implicite Risk Parity** : Pour vraie parite, faudrait leverager les bonds

### Prochaines Etapes

#### Notebook Suivant : QC-Py-09

Nous couvrirons des sujets plus avances comme :
- Optimisation de portefeuille (Mean-Variance, Black-Litterman)
- Factor investing
- Portfolio rebalancing avance

### Ressources Complementaires

- [QuantConnect Multi-Asset Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/)
- [Ray Dalio's All Weather Strategy](https://www.bridgewater.com/research-and-insights/the-all-weather-story)
- [Modern Portfolio Theory (Investopedia)](https://www.investopedia.com/terms/m/modernportfoliotheory.asp)
- [Risk Parity Explained](https://www.investopedia.com/terms/r/risk-parity.asp)

---

**Notebook complete. Pret pour QC-Py-09.**